In [ ]:
import ee
import folium
from pprint import pprint

In [ ]:
import sys, os
sys.dont_write_bytecode = True
sys.path.append(os.path.abspath('../'))

In [ ]:
from modules.Collection import getCollection
from modules.Collection import setProperties
from modules.BandNames import getBandNames
from modules.CloudAndShadowMask import *
from modules.Indexes import *
from modules.SMA_NDFI import *
from modules import Map

In [ ]:
# Initialize
ee.Initialize()


In [ ]:
# select a grid cell
gridName = 'SF-23-Y-D'

assetGrid = 'projects/mapbiomas-workspace/AUXILIAR/cartas'

grid = ee.FeatureCollection(assetGrid)\
    .filterMetadata('grid_name', 'equals', gridName)

In [ ]:
Map.addLayer(grid, visParams={"color":"ff0000"}, name='Grid')

In [ ]:
# Sentinel 2 SR collection parameters
collectionId = 'COPERNICUS/S2_SR'

dateStart = '2020-04-01'
dateEnd = '2020-10-30'

cloudCover = 70.0

In [ ]:
# returns a collection containing the specified parameters
collection = getCollection(collectionId,
                           dateStart=dateStart,
                           dateEnd=dateEnd,
                           cloudCover=cloudCover,
                           geometry=grid.geometry())

In [ ]:
def getImagesNames(collection):
    """Helper function to get a list of image names
    
    Parameters
        collection (ee.ImageCollection): collection from where to get the names
    
    Returns:
        list: list of image names
    """
        
    return collection.reduceColumns(ee.Reducer.toList(), ['system:index']) \
        .get('list') \
        .getInfo()

In [ ]:
imageNames = getImagesNames(collection)

pprint(imageNames)

In [ ]:
# returns  a pattern of band names
bands = getBandNames('sentinel2')
print(bands)

In [ ]:
# rename bands
collection = collection.select(bands['bandNames'], bands['newNames'])

In [ ]:
# cloud and shadow masks parameters
cloudFlag = True
cloudScore = True
cloudShadowFlag = True
cloudShadowTdom = True
zScoreThresh = -1
shadowSumThresh = 4000
dilatePixels = 2
cloudHeights = ee.List.sequence(200, 10000, 500)
cloudBand = 'cloudFlagMask'

In [ ]:
# Get cloud and shadow masks
collectionWithMasks = getMasks(collection,
                               cloudFlag=cloudFlag,
                               cloudScore=cloudScore,
                               cloudShadowFlag=cloudShadowFlag,
                               cloudShadowTdom=cloudShadowTdom,
                               zScoreThresh=zScoreThresh,
                               shadowSumThresh=shadowSumThresh,
                               dilatePixels=dilatePixels,
                               cloudHeights=cloudHeights,
                               cloudBand=cloudBand)

In [ ]:
# get collection without clouds
collectionWithoutClouds = collectionWithMasks.map(
    lambda image: image.mask(
        image.select([
                'cloudFlagMask',
                'cloudScoreMask',
                'cloudShadowFlagMask',
                'cloudShadowTdomMask'
            ]).reduce(ee.Reducer.anyNonZero()).eq(0)
    )
)

In [ ]:
# returns a collection with the calculated indexes
indexesCollection = collectionWithoutClouds.map(getNDVI) \
                                           .map(getEVI)

In [ ]:
# returns a collection with the sma and ndfi calculated
smaCollection = collectionWithoutClouds.map(
    lambda image: getSMAFractions(image, ENDMEMBERS_S2))

ndfiCollection = smaCollection.map(getNDFI)

In [ ]:
# create a folium widget map
coordinates = geometry.centroid().coordinates().getInfo()

m = folium.Map(location=[coordinates[1], coordinates[0]], zoom_start=8, tiles='Cartodb dark_matter')

In [ ]:
# rgb layer
visRGB = {"bands":"swir1,nir,red", "gain": "0.08,0.06,0.2", "gamma":"0.65"}

Map.addLayer(m,
         indexesCollection,
         visParams=visRGB,
         name='RGB')